In [1]:
import pandas as pd
import pathlib
import os
import json
import itertools
import numpy as np

root = pathlib.Path("/data/rbg/shared/datasets/TCGA/")

In [3]:
ids = pd.read_json("ids/tcga_cohort.json")
clinical_data_raw = pd.read_json("./clinical-tcga-brca.json").set_index("case_id")
clinical_data = clinical_data_raw.loc[ids["case_id"]]

clinical_data['stage'] = ids["stage"]

def add_outcome(elem):
    vital_status = elem['demographic'].get('vital_status')

    if vital_status == 'Dead':
        patient_death = elem['demographic'].get('days_to_death')
        return [1, int(patient_death)]

    elif vital_status == "Alive":
        follow_ups = elem['follow_ups']
        follow_up_days = [
            f.get('days_to_follow_up')
            for f in follow_ups
            if f.get('days_to_follow_up') is not None
        ]
        diagnosis_days = [
            d.get('days_to_last_follow_up')
            for d in elem['diagnoses']
            if d.get('days_to_last_follow_up') is not None
        ]
        all_days = follow_up_days + diagnosis_days
        date_of_last_checkup = max(all_days) if all_days else None

        return [0, int(date_of_last_checkup)]
    else:
        typing.assert_never("vital status not in Alive/Dead")

        


clinical_data["outcome"] = clinical_data.apply(add_outcome, axis=1)



def remove_sparse_features(X, threshold=0.05):
    non_zero_fraction = (X != 0).sum(axis=0) / X.shape[0]
    keep = non_zero_fraction > threshold
    print(clinical_data_filtered.shape)
    print(clinical_data_filtered.columns)
    return X.loc[:, keep]


def remove_perfect_correlations(X):
    corr_matrix = X.corr().abs()
    upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
    to_drop = [column for column in upper.columns if any(upper[column] == 1)]
    print(clinical_data_filtered.shape)
    return X.drop(columns=to_drop)

# print(clinical_data.shape)
# clinical_data_filtered = remove_sparse_features(clinical_data)
clinical_data_filtered = clinical_data

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
for col in clinical_data_filtered["stage"].to_frame(): 
    clinical_data_filtered[col] = le.fit_transform(clinical_data_filtered[col])
clinical_data_filtered["stage_copy"] = clinical_data_filtered["stage"]

clinical_data_filtered = clinical_data_filtered.reset_index()[["stage", "outcome"]]
print(clinical_data_filtered.shape)
clinical_data_filtered.head()


(131, 2)


,stage,outcome
0,0,"[0, 915]"
1,0,"[0, 3248]"
2,0,"[0, 1179]"
3,0,"[0, 10]"
4,0,"[0, 397]"
